In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import os,signal
# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import *



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "1234"
database = "AAC"
collection = "animals"
shelter = AnimalShelter(username, password, database, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read())

## Debug
print(len(df.to_dict(orient='records')))
print("Index -> ",df.columns)

#########################
# Dashboard Layout / View
#########################

# map and graph layout
layout = \
    {
        'margin':'10px',
        'padding':'10px',
        'width': '50vw', 
        'height': '500px',
        'box-shadow': '8px 9px 10px #b2aecf',
        "border":"1px solid #b2aecf",
        "background-color":"#575551"
    }

app = JupyterDash("AAC Database")

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'hidden'}),
    html.H1('SNHU CS-340 Dashboard Scotty Intondi'),
    html.Img(id="logo-img",src="/assets/Grazioso Salvare Logo.png"),
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            id="drop_list",
            placeholder="Filter Animal Results...",
            options= [
                {'label':"Show All 💾", 'value':"Show All"},
                {'label':"Water 🌊", 'value':'Water'},
                {'label':"Mountain 🏔️", 'value': "Mountain"},
                {'label':"Disaster 🌋", 'value': "Disaster"},
                {'label':"Elderly 👴", 'value': "Elderly"},
            ]
        )
    ]),
    html.Div(id='filtered-id', children= [
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            sort_action="native",
            sort_mode="multi",
            row_selectable="single",
            selected_columns=[],
            selected_rows=[0],
            page_action="native",
            page_current=0,
            page_size=10,
            style_data_conditional=[{'if': {'row_index': 'odd'},'background-color': '#99a1ad', "color":"#fff"}],
            style_header={'background-color': '#b2aecf','font-weight': 'bold', "color":"#fff"})
    ]),
    html.Div(id="",children=html.H3(f'Scotty Intondi Animal Location: 🐶 🐱 {datetime.date.today()}')), 
    html.Br(),
     html.Hr(),
    html.Div(className="container",children=[
     html.Div(
        id='map-id',
        className='col s12 m6',
     ),
    html.Div(
        dcc.Graph(
            style=layout,
            id='graph-id',
            className='col s12 m6',      
    ))])   
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('filtered-id', "children"),
    [Input('drop_list', "value")]
)
def render_table(item):
    filtered_df = pd.DataFrame.from_records(shelter.read(item))
    return [
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in filtered_df.columns
            ],

            data=filtered_df.to_dict('records'),
            sort_action="native",
            sort_mode="multi",
            row_selectable="single",
            selected_columns=[],
            selected_rows=[0],
            page_action="native",
            page_current=0,
            page_size=10,
            style_data_conditional=[{'if': {'row_index': 'odd'},'background-color': '#99a1ad', "color":"#fff"}],
            style_header={'background-color': '#b2aecf','font-weight': 'bold', "color":"#fff"})
    ]

@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")])
   
def update_graphs(view_data):
    dff = pd.DataFrame.from_dict(view_data)
    fig = px.pie(dff, names='outcome_type')
    fig.update_layout(
        title_text='Outcomes of Animals in Austin Animal Center',
        paper_bgcolor="rgba(0,0,0,0)"
    )
    return fig

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data")],
     [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(view_data,index):
    dff = pd.DataFrame.from_dict(view_data)
    animal = dff.iloc
    if index is None:
        row = 0 
    else:
        row = index[0]
          
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style=layout,
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           dl.Marker(position=[animal[row,13],animal[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                  html.Img(src=f"/assets/{"german_shepard" if animal[row,3]=="Dog" else "cat"}.jpg"),
                 html.H1(f"{"Not specified" if animal[row,9] == "" else animal[row,9]}"),
                  html.Hr(),
                 html.P(f"{"He" if animal[row,12] == "Neutered Male" else "She"} is a {animal[row,1]} old {animal[row,5]} {animal[row,4]} {animal[row,3]}. {"He" if animal[row,12] == "Neutered Male" else "She"} { "since has been" if animal[row,11] == "Return to Owner" or animal[row,11] == "Transfer" or animal[row,11] == "Euthanasia" else "is up for" } {f"euthanized due to {str(animal[row,10]).lower()}" if animal[row,11] == "Euthanasia" else str(animal[row,11]).lower()}.")         
              ])            
           ])
        ])
    ]
    

app.run_server(debug=True)
#os.kill(os.getpid(),signal.SIGTERM)

ServerSelectionTimeoutError: nv-desktop-services.apporto.com:32061: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65768dcf6cdadd61e47c67ec, topology_type: Unknown, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 32061) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:32061: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>